In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [6]:
df_train = pd.read_csv('sales_train.csv')
df_test = pd.read_csv('test.csv')
df_items = pd.read_csv('items.csv', index_col='item_id')
df_categories = pd.read_csv('categories.csv', index_col='category_id')
df_shops = pd.read_csv('shops.csv', index_col='shop_id')
df_sub   = pd.read_csv("sample_submission.csv", index_col="ID")

In [7]:
df_train.describe()
# df_categories.describe()
# df_shops.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [8]:
df_train['item_cnt_day'].sort_values(ascending=False)

2909818    2169.0
2326930    1000.0
2864235     669.0
2851091     637.0
2608040     624.0
            ...  
59156        -6.0
940092       -6.0
2048519      -9.0
1835454     -16.0
1057907     -22.0
Name: item_cnt_day, Length: 2935849, dtype: float64

In [9]:
df_sales_train.corr()["item_price"].abs().sort_values().plot.barh();

NameError: name 'df_sales_train' is not defined

In [ ]:
df_train.groupby("item_id").agg({"shop_id": "nunique"}).reset_index().plot.scatter("item_id", "shop_id", figsize=(10, 6), s=10);

### October 2015 sales

In [ ]:
october_sales = df_train.loc[df_train['date_block_num']==33]
october_sales

In [ ]:
october_sales = df_train[df_train.date_block_num==33]

In [ ]:
df_train.drop(df_train.loc[df_train['date_block_num'] != 33 ].index, inplace=True)

df_train

In [ ]:
df_train.groupby(['date_block_num']).agg(['mean'])

In [ ]:
df_train_1 = df_train[df_train.item_cnt_day < 0]

In [ ]:
full = df_train.shape[0]
ref = df_train_1.shape[0]
percentage_refunds = round(ref / full, 5)
print("% of refunds:", percentage_refunds * 100, '%')

In [ ]:
venn2([set((df_train.shop_id.astype(str) + df_train.item_id.astype(str)).unique()),
       set((df_test.shop_id.astype(str)  + df_test.item_id.astype(str)).unique())],
      set_labels = ('Train', 'Test'), set_colors=('darkcyan', 'goldenrod'), alpha = 0.7)
plt.show()

In [ ]:
venn2([set(df_train.item_id.unique()), set(df_test.item_id.unique())],
      set_labels = ('Train items', 'Test items'), set_colors=('darkcyan', 'goldenrod'), alpha = 0.7)
plt.show()


## October X November

In [ ]:
sep_2015 = df_train[df_train.date_block_num==32]
oct_2015 = df_train[df_train.date_block_num==33]

oct15_item_shop = oct_2015.groupby(["shop_id", "item_id"])["item_cnt_day"].sum().reset_index()

In [ ]:
venn2([set(oct_2015.item_id.unique()),
       set(df_test.item_id.unique())],
      set_labels = ('Items oct', 'Items nov'), set_colors=('darkcyan', 'goldenrod'), alpha = 0.7)
plt.show()


In [ ]:
venn3([set(sep_2015.item_id.unique()),
       set(oct_2015.item_id.unique()),
       set(df_test.item_id.unique())],
      set_labels = ('Items sep', 'Items oct', 'Items nov'), set_colors=('darkcyan', 'goldenrod', 'orchid'), alpha = 0.7)
plt.show()


In [ ]:
october_sales.shop_id.nunique()

In [ ]:
october_sales.item_id.nunique()

In [ ]:
october_sales.groupby(["shop_id", "item_id"])[["item_cnt_day"]].agg(["min", "max", "mean", "count", "sum"])

In [ ]:
october_sales_agg = october_sales.groupby(["shop_id", "item_id"])["item_cnt_day"].sum().reset_index()

In [ ]:
df_test


In [ ]:
df_sub.head()

In [ ]:
results = pd.merge(left = df_test,         # Left table for the join
                   right = october_sales_agg, # Right table for the join
                   on=["shop_id", "item_id"], # Common keys
                   how='left')          # Type of join

results.head()

In [ ]:
print("% of missings:", results.item_cnt_day.isna().sum() / len(results) * 100)

In [ ]:
# Fill missings with zeros

results.item_cnt_day = results.item_cnt_day.fillna(0)
results.head()

In [ ]:
# clipping (limit) mins and max values
results.item_cnt_day = results.item_cnt_day.clip(lower=0, upper=20)
results.head()

In [ ]:
df_sub["item_cnt_month"] = results["item_cnt_day"]
df_sub.head()

In [ ]:
df_sub.to_csv("oct2015.csv")

In [ ]:
!head oct2015.csv